In [1]:
# Add src folder to Python path so db_config can be imported
import sys
import os

sys.path.append(os.path.abspath("../src"))

In [2]:
# Used to execute SQL queries and show results
from sqlalchemy import text
from db_config import engine
import pandas as pd

## **checking for null values**

In [3]:
# Returns NULL counts as a Pandas DataFrame for clarity
def check_nulls(table_name):
    with engine.connect() as conn:
        columns = conn.execute(text("""
            SELECT column_name
            FROM information_schema.columns
            WHERE table_schema = DATABASE()
              AND table_name = :table
        """), {"table": table_name}).fetchall()

        queries = []
        for col in columns:
            queries.append(
                f"SELECT '{col[0]}' AS column_name, "
                f"SUM(CASE WHEN `{col[0]}` IS NULL THEN 1 ELSE 0 END) AS null_count "
                f"FROM `{table_name}`"
            )

        final_query = " UNION ALL ".join(queries)
        result = conn.execute(text(final_query)).fetchall()

    return pd.DataFrame(result, columns=["column_name", "null_count"])


In [4]:
# NULL counts for daily_activity
check_nulls("dailyactivity_merged")

,column_name,null_count
0,ActivityDate,0
1,Calories,0
2,FairlyActiveMinutes,0
3,Id,0
4,LightActiveDistance,0
5,LightlyActiveMinutes,0
6,LoggedActivitiesDistance,0
7,ModeratelyActiveDistance,0
8,SedentaryActiveDistance,0
9,SedentaryMinutes,0


In [5]:
check_nulls("dailycalories_merged")

,column_name,null_count
0,ActivityDay,0
1,Calories,0
2,Id,0


In [6]:
check_nulls("dailyintensities_merged")

,column_name,null_count
0,ActivityDay,0
1,FairlyActiveMinutes,0
2,Id,0
3,LightActiveDistance,0
4,LightlyActiveMinutes,0
5,ModeratelyActiveDistance,0
6,SedentaryActiveDistance,0
7,SedentaryMinutes,0
8,VeryActiveDistance,0
9,VeryActiveMinutes,0


In [7]:
check_nulls("dailysteps_merged")

,column_name,null_count
0,ActivityDay,0
1,Id,0
2,StepTotal,0


In [8]:
check_nulls("heartrate_seconds_merged")

,column_name,null_count
0,Id,0
1,Time,0
2,Value,0


In [9]:
check_nulls('hourlycalories_merged')

,column_name,null_count
0,ActivityHour,0
1,Calories,0
2,Id,0


In [10]:
check_nulls('hourlyintensities_merged')

,column_name,null_count
0,ActivityHour,0
1,AverageIntensity,0
2,Id,0
3,TotalIntensity,0


In [11]:
check_nulls('hourlysteps_merged')

,column_name,null_count
0,ActivityHour,0
1,Id,0
2,StepTotal,0


In [12]:
check_nulls('minuteCaloriesNarrow_merged')

,column_name,null_count
0,ActivityMinute,0
1,Calories,0
2,Id,0


In [13]:
check_nulls('minuteCaloriesWide_merged')

,column_name,null_count
0,ActivityHour,0
1,Calories00,0
2,Calories01,0
3,Calories02,0
4,Calories03,0
...,...,...
57,Calories56,0
58,Calories57,0
59,Calories58,0
60,Calories59,0


In [14]:
check_nulls('minuteIntensitiesNarrow_merged')

,column_name,null_count
0,ActivityMinute,0
1,Id,0
2,Intensity,0


In [15]:
check_nulls('minuteIntensitiesWide_merged')

,column_name,null_count
0,ActivityHour,0
1,Id,0
2,Intensity00,0
3,Intensity01,0
4,Intensity02,0
...,...,...
57,Intensity55,0
58,Intensity56,0
59,Intensity57,0
60,Intensity58,0


In [16]:
check_nulls('minuteMETsNarrow_merged')

,column_name,null_count
0,ActivityMinute,0
1,Id,0
2,METs,0


In [17]:
check_nulls('minutesleep_merged')

,column_name,null_count
0,date,0
1,Id,0
2,logId,0
3,value,0


In [18]:
check_nulls('minuteStepsNarrow_merged')

,column_name,null_count
0,ActivityMinute,0
1,Id,0
2,Steps,0


In [19]:
check_nulls('minuteStepsWide_merged')

,column_name,null_count
0,ActivityHour,0
1,Id,0
2,Steps00,0
3,Steps01,0
4,Steps02,0
...,...,...
57,Steps55,0
58,Steps56,0
59,Steps57,0
60,Steps58,0


In [20]:
check_nulls('sleepday_merged')

,column_name,null_count
0,Id,0
1,SleepDay,0
2,TotalMinutesAsleep,0
3,TotalSleepRecords,0
4,TotalTimeInBed,0


In [21]:

check_nulls('weightloginfo_merged')

,column_name,null_count
0,BMI,0
1,Date,0
2,Fat,65
3,Id,0
4,IsManualReport,0
5,LogId,0
6,WeightKg,0
7,WeightPounds,0


## RESULT
>* NULL profiling shows that all minute-level activity and sleep tables contain complete data with no missing values.
>* Missing values are present only in the `weightloginfo_merged` table, specifically in the `Fat` column.


In [22]:
# Replace NULL fat percentage with 0
with engine.connect() as conn:
    conn.execute(text("""
        UPDATE weightloginfo_merged
        SET Fat = 0
        WHERE Fat IS NULL;
    """))
    conn.commit()

In [23]:
check_nulls('weightloginfo_merged')

,column_name,null_count
0,BMI,0
1,Date,0
2,Fat,0
3,Id,0
4,IsManualReport,0
5,LogId,0
6,WeightKg,0
7,WeightPounds,0


---
---

## **Duplicate Value Checking**

>* Duplicate checks are performed on logical primary keys such as Id + Date/Time combinations to identify repeated records.

In [24]:
# Check duplicate daily activity records per user per date
with engine.connect() as conn:
    conn.execute(text("""
        SELECT Id, ActivityDate, COUNT(*) AS duplicate_count
        FROM dailyactivity_merged
        GROUP BY Id, ActivityDate
        HAVING COUNT(*) > 1;
    """)).fetchall()

In [25]:
# Check duplicate sleep records per user per day
with engine.connect() as conn:
    conn.execute(text("""
        SELECT Id, SleepDay, COUNT(*) AS duplicate_count
        FROM sleepday_merged
        GROUP BY Id, SleepDay
        HAVING COUNT(*) > 1;
    """)).fetchall()


In [26]:
# Check duplicate heart rate records per user per second
with engine.connect() as conn:
    conn.execute(text("""
        SELECT Id, Time, COUNT(*) AS duplicate_count
        FROM heartrate_seconds_merged
        GROUP BY Id, Time
        HAVING COUNT(*) > 1;
    """)).fetchall()


In [27]:
# Check duplicate minute-level sleep records
with engine.connect() as conn:
    conn.execute(text("""
        SELECT Id, date, COUNT(*) AS duplicate_count
        FROM minutesleep_merged
        GROUP BY Id, date
        HAVING COUNT(*) > 1;
    """)).fetchall()


In [28]:
# Check duplicate weight log entries
with engine.connect() as conn:
    conn.execute(text("""
        SELECT Id, Date, COUNT(*) AS duplicate_count
        FROM weightloginfo_merged
        GROUP BY Id, Date
        HAVING COUNT(*) > 1;
    """)).fetchall()


>* no duplicate found in above table (fact table)

## **Data Type Conversion**

>* After handling NULL values and duplicates, column data types were validated and standardized to ensure schema consistency.

In [29]:
# Check column data types for dailyactivity_merged
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = DATABASE()
          AND table_name = 'dailyactivity_merged'
        ORDER BY ordinal_position;
    """)).fetchall()

pd.DataFrame(result, columns=["column_name", "data_type"])

,column_name,data_type
0,Id,bigint
1,ActivityDate,text
2,TotalSteps,bigint
3,TotalDistance,double
4,TrackerDistance,double
5,LoggedActivitiesDistance,double
6,VeryActiveDistance,double
7,ModeratelyActiveDistance,double
8,LightActiveDistance,double
9,SedentaryActiveDistance,double


In [30]:
# Convert ActivityDate values from text to proper DATE format
with engine.connect() as conn:
    conn.execute(text("""
        UPDATE dailyactivity_merged
        SET ActivityDate = STR_TO_DATE(ActivityDate, '%m/%d/%Y');
    """))
    conn.commit()


In [31]:
# Change column type after data conversion
with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE dailyactivity_merged
        MODIFY ActivityDate DATE;
    """))
    conn.commit()


In [32]:
# Validate conversion
with engine.connect() as conn:
    res=conn.execute(text("""
        SELECT ActivityDate
        FROM dailyactivity_merged
        LIMIT 5;
    """)).fetchall()
res

[(datetime.date(2016, 4, 12),),
 (datetime.date(2016, 4, 13),),
 (datetime.date(2016, 4, 14),),
 (datetime.date(2016, 4, 15),),
 (datetime.date(2016, 4, 16),)]

---

In [33]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = DATABASE()
          AND table_name = 'dailycalories_merged'
        ORDER BY ordinal_position;
    """)).fetchall()

pd.DataFrame(result, columns=["column_name", "data_type"])

,column_name,data_type
0,Id,bigint
1,ActivityDay,text
2,Calories,bigint


In [34]:
# Normalize ActivityDay values
with engine.connect() as conn:
    conn.execute(text("""
        UPDATE dailycalories_merged
        SET ActivityDay = STR_TO_DATE(ActivityDay, '%m/%d/%Y');
    """))
    conn.commit()


In [35]:
# Convert ActivityDay to DATE
with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE dailycalories_merged
        MODIFY ActivityDay DATE;
    """))
    conn.commit()


In [36]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = DATABASE()
          AND table_name = 'dailycalories_merged'
        ORDER BY ordinal_position;
    """)).fetchall()

pd.DataFrame(result, columns=["column_name", "data_type"])

,column_name,data_type
0,Id,bigint
1,ActivityDay,date
2,Calories,bigint


---

In [37]:
# Normalize ActivityDay values
with engine.connect() as conn:
    conn.execute(text("""
        UPDATE dailyintensities_merged
        SET ActivityDay = STR_TO_DATE(ActivityDay, '%m/%d/%Y');
    """))
    conn.commit()

In [38]:
# Convert ActivityDay to DATE
with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE dailyintensities_merged
        MODIFY ActivityDay DATE;
    """))
    conn.commit()


In [39]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = DATABASE()
          AND table_name = 'dailyintensities_merged'
        ORDER BY ordinal_position;
    """)).fetchall()

pd.DataFrame(result, columns=["column_name", "data_type"])

,column_name,data_type
0,Id,bigint
1,ActivityDay,date
2,SedentaryMinutes,bigint
3,LightlyActiveMinutes,bigint
4,FairlyActiveMinutes,bigint
5,VeryActiveMinutes,bigint
6,SedentaryActiveDistance,double
7,LightActiveDistance,double
8,ModeratelyActiveDistance,double
9,VeryActiveDistance,double


---

In [40]:
# Normalize ActivityDay values
with engine.connect() as conn:
    conn.execute(text("""
        UPDATE dailysteps_merged
        SET ActivityDay = STR_TO_DATE(ActivityDay, '%m/%d/%Y');
    """))
    conn.commit()


In [41]:
# Convert ActivityDay to DATE
with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE dailysteps_merged
        MODIFY ActivityDay DATE;
    """))
    conn.commit()


In [42]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = DATABASE()
          AND table_name = 'dailyintensities_merged'
        ORDER BY ordinal_position;
    """)).fetchall()

pd.DataFrame(result, columns=["column_name", "data_type"])

,column_name,data_type
0,Id,bigint
1,ActivityDay,date
2,SedentaryMinutes,bigint
3,LightlyActiveMinutes,bigint
4,FairlyActiveMinutes,bigint
5,VeryActiveMinutes,bigint
6,SedentaryActiveDistance,double
7,LightActiveDistance,double
8,ModeratelyActiveDistance,double
9,VeryActiveDistance,double


---

In [43]:
# Normalize SleepDay values including time and AM/PM
with engine.connect() as conn:
    conn.execute(text("""
        UPDATE sleepday_merged
        SET SleepDay = STR_TO_DATE(SleepDay, '%m/%d/%Y %h:%i:%s %p');
    """))
    conn.commit()


In [44]:
# Convert SleepDay to DATE
with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE sleepday_merged
        MODIFY SleepDay DATE;
    """))
    conn.commit()


In [45]:
with engine.connect() as conn:
    res =conn.execute(text("""
        SELECT SleepDay
        FROM sleepday_merged
        LIMIT 5;
    """)).fetchall()
res

[(datetime.date(2016, 4, 12),),
 (datetime.date(2016, 4, 13),),
 (datetime.date(2016, 4, 15),),
 (datetime.date(2016, 4, 16),),
 (datetime.date(2016, 4, 17),)]

---

In [46]:
# Normalize minutesleep date values (AM/PM format)
with engine.connect() as conn:
    conn.execute(text("""
        UPDATE minutesleep_merged
        SET date = STR_TO_DATE(date, '%m/%d/%Y %h:%i:%s %p');
    """))
    conn.commit()


In [47]:
# Convert date to DATETIME
with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE minutesleep_merged
        MODIFY date DATETIME;
    """))
    conn.commit()


In [48]:
with engine.connect() as conn:
    res=conn.execute(text("""
        SELECT date
        FROM minutesleep_merged
        LIMIT 5;
    """)).fetchall()
res

[(datetime.datetime(2016, 4, 12, 2, 47, 30),),
 (datetime.datetime(2016, 4, 12, 2, 48, 30),),
 (datetime.datetime(2016, 4, 12, 2, 49, 30),),
 (datetime.datetime(2016, 4, 12, 2, 50, 30),),
 (datetime.datetime(2016, 4, 12, 2, 51, 30),)]

---

In [49]:
# Check column data types for dailyactivity_merged
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = DATABASE()
          AND table_name = 'heartrate_seconds_merged'
        ORDER BY ordinal_position;
    """)).fetchall()

pd.DataFrame(result, columns=["column_name", "data_type"])

,column_name,data_type
0,Id,bigint
1,Time,text
2,Value,bigint
